# 구글 드라이브에 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# 데이터 호출

In [ ]:
import pandas as pd

df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/WMI_Wafer_DefectPattern_Detection_AnomalyAnalysis/data/LSWMD.pkl")
print(df.shape)
print(df["failureType"].head())      # 보통 list 형태거나 빈 list
print(df["trianTestLabel"].head())   # 기존 train/test 표기가 있을 수 있음(우리는 재분할 권장)

# 데이터 구조 한 번 더 확인

In [ ]:
print(df.columns)
print(df[["waferMap","failureType","trianTestLabel"]].head(3))

# waferMap 한 개의 형태 확인
wm0 = df["waferMap"].iloc[0]
print(type(wm0), getattr(wm0, "shape", None))

# 라벨 컬럼 만들기 (failureType가 list인 경우 처리)

In [ ]:
import numpy as np

def get_label(x):
    if isinstance(x, (list, tuple, np.ndarray)):
        return str(x[0][0]) if len(x) > 0 else None
    if isinstance(x, str) and x.strip() != "":
        return x
    return None

df["label"] = df["failureType"].apply(get_label)

print(df["label"].value_counts(dropna=False).head(15))
print("labeled count:", df["label"].notna().sum())

In [ ]:
print(type(df.iloc[0]["label"]))

# 라벨 있는 데이터만 필터링 + Unknown 클래스 홀드아웃 만들기
Donut과 Scratch를 Unknown으로 만들고 진행

In [ ]:
UNKNOWN_CLASSES = ["Donut", "Scratch"]

labeled = df[df["label"].notna()].copy()
known_df = labeled[~labeled["label"].isin(UNKNOWN_CLASSES)].copy()
unknown_df = labeled[labeled["label"].isin(UNKNOWN_CLASSES)].copy()

print("labeled:", len(labeled))
print("known:", len(known_df))
print("unknown:", len(unknown_df))
print("known classes:", sorted(known_df["label"].unique()))
print("unknown classes:", sorted(unknown_df["label"].unique()))

### 클래스 별 대표 1장씩 그리드로 보기

## 각 클래스 별로 그리드로 보기

- **Center**: 웨이퍼 **중심(가운데)**에 불량이 몰려 있는 패턴 (가운데 뭉침)
- **Donut**: 중심은 비교적 깨끗하고, 그 주변에 **동심원(도넛 링)**처럼 불량이 분포 (원형 띠)
- **Edge-Loc (Edge-Local)**: 웨이퍼 **가장자리의 특정 구간**에만 불량이 몰린 패턴 (테두리 한쪽만)
- **Edge-Ring**: 웨이퍼 **테두리 전체를 따라 고리(링)**처럼 불량이 나타나는 패턴 (둘레 전체 띠)
- **Loc (Local)**: 웨이퍼 내부에서 **중심/테두리가 아닌 임의 위치**에 불량이 뭉친 패턴 (안쪽 어딘가 덩어리)
- **Random**: 웨이퍼 전체에 불량이 **듬성듬성, 비교적 고르게 흩어진** 패턴 (여기저기 점)
- **Scratch**: **직선/곡선 형태로 길게 이어진** 불량 패턴 (긁힌 자국처럼 선/호)
- **Near-full**: 웨이퍼 **거의 전체가 불량으로 채워진** 패턴 (대부분 불량)
- **none**: 뚜렷한 결함 패턴이 없거나 결함이 거의 없는 상태 (패턴 없음)
---
- 보라(가장 어두운 색) = 작은 값 (보통 0) → 웨이퍼 바깥/배경
- 파랑(중간 값) = 중간 값 (보통 1) → 정상 die (pass)
- 노랑(가장 밝은 색) = 큰 값 (보통 2) → 불량 die (fail)

In [ ]:
import matplotlib.pyplot as plt

labels = sorted(labeled["label"].unique())
n = len(labels)

cols = 4
rows = (n + cols - 1) // cols

plt.figure(figsize=(4*cols, 4*rows))
for i, lab in enumerate(labels):
    wm = labeled[labeled["label"] == lab]["waferMap"].iloc[0]  # 첫 샘플
    ax = plt.subplot(rows, cols, i+1)
    ax.imshow(wm, interpolation="nearest")
    ax.set_title(lab)
    ax.axis("off")
plt.tight_layout()
plt.show()

### 특정 Failure Type에서 여러 장 랜덤 샘플 보기

In [ ]:
import matplotlib.pyplot as plt

def show_samples(label, n=8, seed=42):
    sub = labeled[labeled["label"] == label]
    if len(sub) == 0:
        print("No samples for:", label)
        return
    sub = sub.sample(n=min(n, len(sub)), random_state=seed)

    cols = 4
    rows = (len(sub) + cols - 1) // cols
    plt.figure(figsize=(4*cols, 4*rows))
    for i, wm in enumerate(sub["waferMap"].tolist()):
        ax = plt.subplot(rows, cols, i+1)
        ax.imshow(wm, interpolation="nearest")
        ax.axis("off")
    plt.suptitle(f"{label} (n={len(sub)})", y=1.02, fontsize=14)
    plt.tight_layout()
    plt.show()

# 예시
for label in ["Center", "Donut", "Edge-Loc", "Edge-Ring", "Loc", "Random", "Scratch", "Near-full", "none"]:
    show_samples(label, n=8, seed=0)

# Train/Val/Test 분할 (Known만 stratify로 분할)
- train = 70%
- val = 15%
- test_known = 15%
- test_unknown = unknown_df 전부

이 비율은 머신러닝에서 아주 흔한 기본값

- train을 충분히 크게 가져가야 학습이 잘 되고
- val/test도 너무 작으면 지표가 흔들리니까 적당히 확보하는 균형

In [ ]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(
    known_df, test_size=0.3, random_state=42, stratify=known_df["label"] # stratify는 각 split마다 라벨 비율이 원래 데이터랑 비슷하게 유지되도록 나눠주는 옵션
)
val_df, test_known_df = train_test_split(
    temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"]
)

test_unknown_df = unknown_df.copy()

print("train:", len(train_df), "val:", len(val_df),
      "test_known:", len(test_known_df), "test_unknown:", len(test_unknown_df))

# PyTorch Dataset/DataLoader 만들기 (MVP용 간단 전처리)

*   waferMap을 64×64로 resize
*   1채널을 3채널로 복제해서 ResNet에 넣기 (가장 쉬운 방식)
*   회전/반전 증강은 train에만



In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

known_classes = sorted(train_df["label"].unique())
class_to_idx = {c:i for i,c in enumerate(known_classes)}
idx_to_class = {i:c for c,i in class_to_idx.items()}
print(class_to_idx)

class WaferMapDataset(Dataset):
    def __init__(self, df, class_to_idx=None, is_train=False, resize=64):
        self.df = df.reset_index(drop=True)
        self.class_to_idx = class_to_idx
        self.is_train = is_train
        self.resize = resize

    def __len__(self):
        return len(self.df)

    def _augment(self, x):
        # x: (1,H,W)
        if torch.rand(1).item() < 0.5:
            x = torch.flip(x, dims=[2])  # 좌우
        if torch.rand(1).item() < 0.5:
            x = torch.flip(x, dims=[1])  # 상하
        k = int(torch.randint(0, 4, (1,)).item())
        x = torch.rot90(x, k, dims=[1,2])
        return x

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        wm = row["waferMap"]  # numpy 2D
        x = torch.tensor(wm, dtype=torch.float32).unsqueeze(0)  # (1,H,W)

        # 0/1/2 형태면 0~1로 스케일 (선택)
        mx = x.max()
        x = x / (mx if mx > 0 else 1.0)

        # resize
        x = F.interpolate(x.unsqueeze(0), size=(self.resize, self.resize), mode="nearest").squeeze(0)

        if self.is_train:
            x = self._augment(x)

        # ResNet 입력 맞추기: 3채널
        x = x.repeat(3, 1, 1)  # (3,resize,resize)

        if self.class_to_idx is None:
            return x

        y = self.class_to_idx[row["label"]]
        return x, y

train_loader = DataLoader(WaferMapDataset(train_df, class_to_idx, True, 64),
                          batch_size=128, shuffle=True, num_workers=2)
val_loader = DataLoader(WaferMapDataset(val_df, class_to_idx, False, 64),
                        batch_size=256, shuffle=False, num_workers=2)
test_known_loader = DataLoader(WaferMapDataset(test_known_df, class_to_idx, False, 64),
                               batch_size=256, shuffle=False, num_workers=2)
test_unknown_loader = DataLoader(
    WaferMapDataset(test_unknown_df, class_to_idx=None, is_train=False, resize=64),
    batch_size=256, shuffle=False, num_workers=2
)

# Known 분류기 학습 (ResNet18 베이스라인)

In [ ]:
import torchvision.models as models
import torch.nn as nn
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, len(known_classes))
model = model.to(device)

opt = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

def run_epoch(loader, train=True):
    model.train(train)
    total_loss, correct, total = 0, 0, 0
    for x, y in tqdm(loader, leave=False):
        x = x.to(device)
        y = torch.tensor(y).to(device)

        logits = model(x)
        loss = criterion(logits, y)

        if train:
            opt.zero_grad()
            loss.backward()
            opt.step()

        total_loss += loss.item() * x.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += x.size(0)

    return total_loss/total, correct/total

for epoch in range(1, 6):
    tr_loss, tr_acc = run_epoch(train_loader, train=True)
    va_loss, va_acc = run_epoch(val_loader, train=False)
    print(f"Epoch {epoch} | train loss {tr_loss:.4f} acc {tr_acc:.4f} | val loss {va_loss:.4f} acc {va_acc:.4f}")

# 학습된 모델 저장하기

모델의 가중치(`state_dict`)를 저장합니다. 일반적으로 `.pth` 또는 `.pt` 확장자를 사용합니다.

In [ ]:
import os
import json
from datetime import datetime
import torch

# 1) 저장 폴더(Drive)
CKPT_DIR = "/content/drive/MyDrive/Colab Notebooks/WMI_Wafer_DefectPattern_Detection_AnomalyAnalysis/checkpoints"
os.makedirs(CKPT_DIR, exist_ok=True)

def _safe(s: str) -> str:
    # 파일명 안전하게(공백/슬래시 등 제거)
    return (
        str(s)
        .replace(" ", "")
        .replace("/", "-")
        .replace("\\", "-")
        .replace(",", "-")
        .replace("[", "")
        .replace("]", "")
        .replace("'", "")
        .replace('"', "")
    )

def make_run_name(
    arch: str,
    resize: int,
    unknown_classes,
    epochs: int = None,
    val_macro_f1: float = None,
    auroc_msp: float = None,
    auroc_energy: float = None,
):
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    holdout = "-".join([_safe(x) for x in unknown_classes]) if unknown_classes else "none"
    parts = [
        ts,
        _safe(arch),
        f"r{resize}",
        f"holdout-{holdout}",
    ]
    if epochs is not None:
        parts.append(f"ep{int(epochs):02d}")
    if val_macro_f1 is not None:
        parts.append(f"valF1-{val_macro_f1:.3f}")
    if auroc_msp is not None:
        parts.append(f"aurocMSP-{auroc_msp:.3f}")
    if auroc_energy is not None:
        parts.append(f"aurocEn-{auroc_energy:.3f}")

    return "_".join(parts)

def save_checkpoint(
    path: str,
    model,
    class_to_idx: dict,
    unknown_classes: list,
    resize: int,
    arch: str = "resnet18",
    epochs: int = None,
    metrics: dict = None,
    extra_cfg: dict = None,
    optimizer=None,   # resume까지 원하면 전달
):
    ckpt = {
        "arch": arch,
        "resize": resize,
        "unknown_classes": unknown_classes,
        "class_to_idx": class_to_idx,
        "model_state_dict": model.state_dict(),
        "metrics": metrics or {},
        "cfg": extra_cfg or {},
        "saved_at": datetime.now().isoformat(),
    }
    if optimizer is not None:
        ckpt["optimizer_state_dict"] = optimizer.state_dict()

    torch.save(ckpt, path)
    print(f"✅ Saved checkpoint: {path}")
    # 사람이 보기 좋게 메타도 같이 저장(선택)
    meta_path = path.replace(".pt", ".json")
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump({k: ckpt[k] for k in ["arch","resize","unknown_classes","metrics","cfg","saved_at"]}, f, ensure_ascii=False, indent=2)
    print(f"📝 Saved meta: {meta_path}")


# Known 분류 성능을 “macro-F1”로 평가 (중요)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, classification_report

@torch.no_grad()
def predict_known(loader):
    model.eval()
    ys, ps = [], []
    for x, y in loader:
        x = x.to(device)
        logits = model(x)
        pred = logits.argmax(dim=1).cpu().numpy()
        ys.append(np.array(y))
        ps.append(pred)
    y_true = np.concatenate(ys)
    y_pred = np.concatenate(ps)
    return y_true, y_pred

y_true, y_pred = predict_known(test_known_loader)
print("macro-F1:", f1_score(y_true, y_pred, average="macro"))
print(classification_report(y_true, y_pred, target_names=[idx_to_class[i] for i in range(len(known_classes))]))

# Unknown 탐지(OOD): MSP vs Energy로 AUROC/AUPR 계산

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score

@torch.no_grad()
def collect_scores(loader, known=True, T=1.0):
    model.eval()
    msp_list, energy_list = [], []
    for batch in loader:
        x = batch[0] if known else batch
        x = x.to(device)

        logits = model(x) / T
        prob = torch.softmax(logits, dim=1)
        msp = prob.max(dim=1).values                   # 높을수록 known
        energy = -T * torch.logsumexp(logits, dim=1)   # 보통 -energy가 높을수록 known로 사용

        msp_list.append(msp.cpu().numpy())
        energy_list.append(energy.cpu().numpy())

    return np.concatenate(msp_list), np.concatenate(energy_list)

msp_k, en_k = collect_scores(test_known_loader, known=True)
msp_u, en_u = collect_scores(test_unknown_loader, known=False)

y = np.concatenate([np.ones_like(msp_k), np.zeros_like(msp_u)])  # known=1, unknown=0

score_msp = np.concatenate([msp_k, msp_u])
score_energy = np.concatenate([-en_k, -en_u])  # -energy를 known 점수로

print("AUROC MSP   :", roc_auc_score(y, score_msp))
print("AUPR  MSP   :", average_precision_score(y, score_msp))
print("AUROC Energy:", roc_auc_score(y, score_energy))
print("AUPR  Energy:", average_precision_score(y, score_energy))

# 저장된 모델 불러오기

저장된 `state_dict`를 불러와 새로운 모델 인스턴스에 로드합니다. 이때, 모델의 아키텍처는 저장할 때와 동일하게 정의되어 있어야 합니다.

In [ ]:
import os, json
from datetime import datetime
import torch
import torchvision.models as models
import torch.nn as nn

CKPT_DIR = "/content/drive/MyDrive/Colab Notebooks/WMI_Wafer_DefectPattern_Detection_AnomalyAnalysis/checkpoints"
os.makedirs(CKPT_DIR, exist_ok=True)

def _safe(s: str) -> str:
    return (
        str(s).replace(" ", "")
        .replace("/", "-").replace("\\", "-")
        .replace(",", "-").replace("[", "").replace("]", "")
        .replace("'", "").replace('"', "")
    )

def _get(name, default=None):
    # 노트북 전역 변수에서 자동으로 가져오기
    return globals().get(name, default)

def save_mvp_checkpoint(
    model,
    class_to_idx,
    unknown_classes,
    resize=64,
    arch="resnet18",
    optimizer=None,
    ckpt_dir=CKPT_DIR,
    extra_cfg=None,
):
    """
    ✅ 자동 저장:
    - 노트북에 val_macro_f1 / auroc_msp / auroc_energy / epochs 등의 변수가 있으면 자동으로 메타+파일명에 반영
    - 없으면 없는대로 저장 (MVP 중간 저장에도 사용 가능)
    """
    os.makedirs(ckpt_dir, exist_ok=True)

    # 자동으로 메트릭/설정 값 끌어오기(있으면 쓰고, 없으면 None)
    epochs = _get("epochs_run", None) or _get("EPOCHS", None) or _get("epochs", None)
    val_macro_f1 = _get("val_macro_f1", None)
    auroc_msp = _get("auroc_msp", None)
    auroc_energy = _get("auroc_energy", None)
    holdout = "-".join([_safe(x) for x in (unknown_classes or [])]) or "none"
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")

    # 파일명 구성(있는 정보만 추가)
    parts = [ts, _safe(arch), f"r{int(resize)}", f"holdout-{holdout}"]
    if epochs is not None:
        parts.append(f"ep{int(epochs):02d}")
    if val_macro_f1 is not None:
        parts.append(f"valF1-{float(val_macro_f1):.3f}")
    if auroc_msp is not None:
        parts.append(f"aurocMSP-{float(auroc_msp):.3f}")
    if auroc_energy is not None:
        parts.append(f"aurocEn-{float(auroc_energy):.3f}")

    run_name = "_".join(parts)
    ckpt_path = os.path.join(ckpt_dir, run_name + ".pt")

    # 체크포인트 내용
    ckpt = {
        "arch": arch,
        "resize": int(resize),
        "unknown_classes": list(unknown_classes) if unknown_classes is not None else None,
        "class_to_idx": dict(class_to_idx),
        "model_state_dict": model.state_dict(),
        "metrics": {
            "val_macro_f1": val_macro_f1,
            "auroc_msp": auroc_msp,
            "auroc_energy": auroc_energy,
        },
        "cfg": extra_cfg or {},
        "saved_at": datetime.now().isoformat(),
    }
    if optimizer is not None:
        ckpt["optimizer_state_dict"] = optimizer.state_dict()

    torch.save(ckpt, ckpt_path)

    # 사람이 보기 좋은 메타 json도 같이 저장(선택이지만 추천)
    meta_path = ckpt_path.replace(".pt", ".json")
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(
            {k: ckpt[k] for k in ["arch", "resize", "unknown_classes", "metrics", "cfg", "saved_at"]},
            f, ensure_ascii=False, indent=2
        )

    print("✅ Saved:", ckpt_path)
    print("📝 Meta :", meta_path)
    return ckpt_path

def load_mvp_checkpoint(path, device="cpu"):
    ckpt = torch.load(path, map_location=device)

    class_to_idx = ckpt["class_to_idx"]
    known_classes = [None] * len(class_to_idx)
    for cls, idx in class_to_idx.items():
        known_classes[idx] = cls

    arch = ckpt.get("arch", "resnet18")
    if arch == "resnet18":
        model = models.resnet18(weights=None)
        model.fc = nn.Linear(model.fc.in_features, len(known_classes))
    else:
        raise ValueError(f"Unsupported arch: {arch}")

    model.load_state_dict(ckpt["model_state_dict"])
    model = model.to(device).eval()

    print("✅ Loaded:", path)
    print(" - arch:", arch, "| resize:", ckpt.get("resize"))
    print(" - unknown_classes:", ckpt.get("unknown_classes"))
    print(" - metrics:", ckpt.get("metrics", {}))
    return model, ckpt, known_classes, class_to_idx

In [ ]:
ckpt_path = save_mvp_checkpoint(model, class_to_idx, UNKNOWN_CLASSES, resize=64, arch="resnet18", optimizer=None)

In [ ]:
ckpt_path = "/content/drive/MyDrive/Colab Notebooks/WMI_Wafer_DefectPattern_Detection_AnomalyAnalysis/checkpoints/20251231-152621_resnet18_r64_holdout-Donut-Scratch_ep05_valF1-0.872_aurocMSP-0.862_aurocEn-0.852.pt"
device = "cuda" if torch.cuda.is_available() else "cpu"
loaded_model, ckpt, known_classes_loaded, class_to_idx_loaded = load_mvp_checkpoint(ckpt_path, device=device)

# 트리아지용 Dataset: "이미지 + 라벨 + row index" 같이 반환

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np

class WaferMapDatasetWithIdx(Dataset):
    def __init__(self, df, class_to_idx=None, is_train=False, resize=64):
        self.df = df.reset_index(drop=True).copy()
        self.class_to_idx = class_to_idx
        self.is_train = is_train
        self.resize = resize

    def __len__(self):
        return len(self.df)

    def _augment(self, x):
        if torch.rand(1).item() < 0.5:
            x = torch.flip(x, dims=[2])  # 좌우
        if torch.rand(1).item() < 0.5:
            x = torch.flip(x, dims=[1])  # 상하
        k = int(torch.randint(0, 4, (1,)).item())
        x = torch.rot90(x, k, dims=[1,2])
        return x

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        wm = row["waferMap"]
        x = torch.tensor(wm, dtype=torch.float32).unsqueeze(0)  # (1,H,W)

        mx = x.max()
        x = x / (mx if mx > 0 else 1.0)

        x = F.interpolate(x.unsqueeze(0), size=(self.resize, self.resize), mode="nearest").squeeze(0)

        if self.is_train:
            x = self._augment(x)

        x = x.repeat(3, 1, 1)  # (3,resize,resize)

        # 라벨이 없으면 x, idx만
        if self.class_to_idx is None:
            return x, idx

        y = self.class_to_idx[row["label"]]
        return x, y, idx

# 레퍼런스(known) 풀을 “적당히 샘플링”해서 준비

In [ ]:
# ✅ 추천: reference에서는 none을 줄이거나 제외
MAX_PER_CLASS = 1500
MAX_NONE = 2000

ref_parts = []
for c in known_classes:
    sub = train_df[train_df["label"] == c]
    if c == "none":
        sub = sub.sample(n=min(len(sub), MAX_NONE), random_state=42)
    else:
        sub = sub.sample(n=min(len(sub), MAX_PER_CLASS), random_state=42)
    ref_parts.append(sub)

ref_df = pd.concat(ref_parts).sample(frac=1.0, random_state=42).reset_index(drop=True)
print("ref_df size:", len(ref_df))
print(ref_df["label"].value_counts().head())

# 임베딩 추출 모델(ResNet18의 fc 직전)

In [ ]:
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

# model은 너가 학습한 모델(또는 loaded_model) 사용
model = loaded_model if "loaded_model" in globals() else model
model = model.to(device).eval()

# fc 제거 → 임베딩 추출기
embed_model = nn.Sequential(*list(model.children())[:-1]).to(device).eval()

@torch.no_grad()
def collect_embeddings_ref(loader):
    embs, labels, idxs = [], [], []
    for x, y, idx in loader:
        x = x.to(device)
        e = embed_model(x).squeeze(-1).squeeze(-1)  # (N,512,1,1) -> (N,512)
        embs.append(e.cpu())
        labels.append(torch.tensor(y))
        idxs.append(torch.tensor(idx))
    return torch.cat(embs, 0).numpy(), torch.cat(labels, 0).numpy(), torch.cat(idxs, 0).numpy()

@torch.no_grad()
def collect_embeddings_unknown(loader):
    embs, idxs = [], []
    for x, idx in loader:
        x = x.to(device)
        e = embed_model(x).squeeze(-1).squeeze(-1)
        embs.append(e.cpu())
        idxs.append(torch.tensor(idx))
    return torch.cat(embs, 0).numpy(), torch.cat(idxs, 0).numpy()

# Loader 만들고 임베딩 뽑기

In [ ]:
# reference(known)
ref_loader = DataLoader(
    WaferMapDatasetWithIdx(ref_df, class_to_idx=class_to_idx, is_train=False, resize=64),
    batch_size=256, shuffle=False, num_workers=2
)

# unknown(test_unknown_df)
unk_loader = DataLoader(
    WaferMapDatasetWithIdx(test_unknown_df, class_to_idx=None, is_train=False, resize=64),
    batch_size=256, shuffle=False, num_workers=2
)

ref_emb, ref_y, ref_local_idx = collect_embeddings_ref(ref_loader)
unk_emb, unk_local_idx = collect_embeddings_unknown(unk_loader)

print("ref_emb:", ref_emb.shape, "unk_emb:", unk_emb.shape)

# Unknown → 유사사례 Top-K 검색(코사인 유사도)

In [ ]:
import numpy as np

def l2norm(a):
    return a / (np.linalg.norm(a, axis=1, keepdims=True) + 1e-12)

ref_n = l2norm(ref_emb)
unk_n = l2norm(unk_emb)

# Top-K retrieval
K = 5
# (unk x ref) similarity가 커질 수 있으니 unk를 배치로 처리
def topk_retrieval(unk_n, ref_n, k=5, chunk=512):
    topk_idx_all, topk_sim_all = [], []
    ref_t = torch.tensor(ref_n, dtype=torch.float32)
    for i in range(0, len(unk_n), chunk):
        u = torch.tensor(unk_n[i:i+chunk], dtype=torch.float32)
        sim = u @ ref_t.T  # cosine similarity
        topk_sim, topk_idx = torch.topk(sim, k=k, dim=1)
        topk_idx_all.append(topk_idx.numpy())
        topk_sim_all.append(topk_sim.numpy())
    return np.vstack(topk_idx_all), np.vstack(topk_sim_all)

topk_idx, topk_sim = topk_retrieval(unk_n, ref_n, k=K, chunk=512)
print(topk_idx.shape, topk_sim.shape)  # (num_unknown, K)

Top-K 결과를 “이미지로” 보여주는 함수

In [ ]:
import matplotlib.pyplot as plt

def show_triage_case(unk_i, K=5):
    # unknown 샘플
    unk_row = test_unknown_df.iloc[int(unk_local_idx[unk_i])]
    unk_wm = unk_row["waferMap"]

    # ref 샘플들
    ref_indices = topk_idx[unk_i][:K]
    ref_sims = topk_sim[unk_i][:K]

    plt.figure(figsize=(3*(K+1), 3))
    # unknown
    plt.subplot(1, K+1, 1)
    plt.imshow(unk_wm)
    plt.title(f"UNKNOWN\n(true={unk_row['label']})")
    plt.axis("off")

    for j in range(K):
        rloc = int(ref_local_idx[ref_indices[j]])  # ref_df의 로컬 인덱스
        rrow = ref_df.iloc[rloc]
        plt.subplot(1, K+1, j+2)
        plt.imshow(rrow["waferMap"])
        plt.title(f"{rrow['label']}\ncos={ref_sims[j]:.3f}")
        plt.axis("off")

    plt.tight_layout()
    plt.show()

# 예시로 3개만 보기
for i in [0, 1, 2]:
    show_triage_case(i, K=5)

# 2D 시각화(UMAP 추천, 없으면 t-SNE)

In [ ]:
!pip -q install umap-learn

import umap
import numpy as np
import matplotlib.pyplot as plt

# 시각화는 너무 많으면 느리니 샘플링
N_REF_VIS = min(5000, len(ref_emb))
N_UNK_VIS = min(2000, len(unk_emb))

rng = np.random.RandomState(42)
ref_vis_idx = rng.choice(len(ref_emb), size=N_REF_VIS, replace=False)
unk_vis_idx = rng.choice(len(unk_emb), size=N_UNK_VIS, replace=False)

X_vis = np.vstack([ref_emb[ref_vis_idx], unk_emb[unk_vis_idx]])
y_vis = np.concatenate([ref_y[ref_vis_idx], -1*np.ones(N_UNK_VIS, dtype=int)])  # unknown=-1

reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
Z = reducer.fit_transform(X_vis)

plt.figure(figsize=(8,6))
plt.scatter(Z[:,0], Z[:,1], c=y_vis, s=6)  # 색은 클래스 id로 자동
plt.title("Embedding 2D (UMAP): known vs unknown")
plt.show()

In [ ]:
import numpy as np
print("known points:", np.sum(y_vis != -1))
print("unknown points:", np.sum(y_vis == -1))
print("unique labels (incl -1):", np.unique(y_vis)[:20])

In [ ]:
import matplotlib.pyplot as plt

known_mask = (y_vis != -1)
unk_mask = (y_vis == -1)

plt.figure(figsize=(8,6))
plt.scatter(Z[known_mask,0], Z[known_mask,1], s=6, alpha=0.25, label="known")
plt.scatter(Z[unk_mask,0],   Z[unk_mask,1],   s=8, alpha=0.9,  label="unknown")
plt.title("Embedding 2D (UMAP): known (gray) vs unknown (red)")
plt.legend()
plt.show()

In [ ]:
import umap
import numpy as np
import matplotlib.pyplot as plt

def l2norm(a):
    return a / (np.linalg.norm(a, axis=1, keepdims=True) + 1e-12)

Xn = l2norm(X_vis)  # 코사인에 맞게 정규화(권장)

reducer = umap.UMAP(metric="cosine", n_neighbors=15, min_dist=0.1, random_state=42)
Z = reducer.fit_transform(Xn)

known_mask = (y_vis != -1)
unk_mask = (y_vis == -1)

plt.figure(figsize=(8,6))
plt.scatter(Z[known_mask,0], Z[known_mask,1], s=6, alpha=0.25, color="gray", label="known")
plt.scatter(Z[unk_mask,0],   Z[unk_mask,1],   s=10, alpha=0.9,  color="red",  label="unknown")
plt.title("Embedding 2D (UMAP-cosine): known (gray) vs unknown (red)")
plt.legend()
plt.show()

# Top-K 트리아지 구현

In [ ]:
import numpy as np
import torch

def l2norm(a):
    return a / (np.linalg.norm(a, axis=1, keepdims=True) + 1e-12)

ref_n = l2norm(ref_emb).astype(np.float32)
unk_n = l2norm(unk_emb).astype(np.float32)

def topk_retrieval(unk_n, ref_n, k=5, chunk=512):
    topk_idx_all, topk_sim_all = [], []
    ref_t = torch.tensor(ref_n)  # (Nref, D)
    for i in range(0, len(unk_n), chunk):
        u = torch.tensor(unk_n[i:i+chunk])   # (chunk, D)
        sim = u @ ref_t.T                    # (chunk, Nref)
        topk_sim, topk_idx = torch.topk(sim, k=k, dim=1)
        topk_idx_all.append(topk_idx.numpy())
        topk_sim_all.append(topk_sim.numpy())
    return np.vstack(topk_idx_all), np.vstack(topk_sim_all)

K = 5
topk_idx, topk_sim = topk_retrieval(unk_n, ref_n, k=K, chunk=512)
print(topk_idx.shape, topk_sim.shape)  # (N_unknown, K)

트리아지 “요약 한 줄” 만들기(발표용 핵심)

In [ ]:
from collections import Counter
import numpy as np

def triage_summary(unk_i, K=5):
    labels = [ref_df.iloc[j]["label"] for j in topk_idx[unk_i][:K]]
    cnt = Counter(labels)
    top_label, top_count = cnt.most_common(1)[0]
    purity = top_count / K
    uniq = len(cnt)

    if purity >= 0.8:
        msg = f"Top-{K} 중 {top_label} {top_count}/{K} → '{top_label}' 계열 가능성 높음"
    elif uniq >= 4:
        msg = f"Top-{K} 라벨 다양({dict(cnt)}) → 애매 케이스(추가 확인/재측정 권장)"
    else:
        msg = f"Top-{K} 혼재({dict(cnt)}) → 유사 패턴 후보 복수"
    return labels, msg

# 화면에 “Unknown + Top-K” 시각화 (데모)

In [ ]:
import matplotlib.pyplot as plt

def show_triage_case(unk_i, K=5):
    unk_row = test_unknown_df.iloc[unk_i]
    unk_wm = unk_row["waferMap"]

    labels, msg = triage_summary(unk_i, K)

    plt.figure(figsize=(3*(K+1), 3))
    plt.subplot(1, K+1, 1)
    plt.imshow(unk_wm)
    plt.title(f"UNKNOWN\n(true={unk_row['label']})")
    plt.axis("off")

    for j in range(K):
        rrow = ref_df.iloc[topk_idx[unk_i][j]]
        plt.subplot(1, K+1, j+2)
        plt.imshow(rrow["waferMap"])
        plt.title(f"{rrow['label']}\ncos={topk_sim[unk_i][j]:.3f}")
        plt.axis("off")

    plt.suptitle(msg, y=1.05, fontsize=12)
    plt.tight_layout()
    plt.show()

# 예시 3개 확인
for i in [0, 1, 2]:
    show_triage_case(i, K=5)

# 데모용 이미지 파일로 저장

In [ ]:
import os
import matplotlib.pyplot as plt

OUT_DIR = "assets/triage"
os.makedirs(OUT_DIR, exist_ok=True)

def save_triage_case(unk_i, K=5):
    unk_row = test_unknown_df.iloc[unk_i]
    labels, msg = triage_summary(unk_i, K)

    plt.figure(figsize=(3*(K+1), 3))
    plt.subplot(1, K+1, 1)
    plt.imshow(unk_row["waferMap"])
    plt.title(f"UNKNOWN\n(true={unk_row['label']})")
    plt.axis("off")

    for j in range(K):
        rrow = ref_df.iloc[topk_idx[unk_i][j]]
        plt.subplot(1, K+1, j+2)
        plt.imshow(rrow["waferMap"])
        plt.title(f"{rrow['label']}\ncos={topk_sim[unk_i][j]:.3f}")
        plt.axis("off")

    plt.suptitle(msg, y=1.05, fontsize=12)
    plt.tight_layout()

    fname = f"triage_unk{unk_i:04d}_true-{unk_row['label']}.png"
    path = os.path.join(OUT_DIR, fname)
    plt.savefig(path, dpi=200, bbox_inches="tight")
    plt.close()
    return path, msg

# 6개만 저장
picked = [0, 1, 2, 3, 4, 5]
for i in picked:
    p, m = save_triage_case(i, K=5)
    print(p, "|", m)

# OOD 점수(MSP/Energy) 다시 계산해서 “데모 후보” 자동 선별

## OOD 점수(MSP/Energy) 다시 계산해서 “데모 후보” 자동 선별
이미 unknown에 대한 MSP/Energy 배열을 갖고 있으면 이 셀은 건너뛰어도 됨.

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

# 전제: model(or loaded_model), device, test_unknown_df, WaferMapDataset(혹은 너가 쓰는 Dataset)이 존재
model_use = loaded_model if "loaded_model" in globals() else model_use if "model_use" in globals() else model
model_use = model_use.to(device).eval()

# unknown loader (순서 보존 위해 shuffle=False)
unk_score_loader = DataLoader(
    WaferMapDataset(test_unknown_df, class_to_idx=None, is_train=False, resize=64),
    batch_size=256, shuffle=False, num_workers=2
)

@torch.no_grad()
def compute_msp_energy(model, loader):
    msps = []
    energy_raw = []  # Energy = -logsumexp(logits). OOD일수록 값이 덜 음수(=더 큼)로 가는 경향
    for batch in loader:
        # loader가 x만 주는 경우 / (x,idx) 주는 경우를 모두 처리
        x = batch[0] if isinstance(batch, (list, tuple)) else batch
        x = x.to(device)
        logits = model(x)

        prob = F.softmax(logits, dim=1)
        msp = prob.max(dim=1).values

        e = -torch.logsumexp(logits, dim=1)  # energy_raw
        msps.append(msp.cpu().numpy())
        energy_raw.append(e.cpu().numpy())

    msps = np.concatenate(msps)
    energy_raw = np.concatenate(energy_raw)
    return msps, energy_raw

msp_u, energy_u = compute_msp_energy(model_use, unk_score_loader)
print("scores:", msp_u.shape, energy_u.shape, "len(df)=", len(test_unknown_df))

## 데모용 Unknown 샘플 자동 선택 (2종류)
*   (A) unknown스러운 샘플: MSP 낮고, Energy_raw는 큰(덜 음수) 샘플
*   (B) 위험 샘플(과신): MSP 높은데도 실제론 unknown인 샘플 (과신 오분류 위험 강조용)



In [ ]:
N_DEMO = 8  # 저장할 케이스 개수

# A) unknown-likely: MSP 낮은 순 + Energy_raw 큰 순을 rank로 합침
rank_msp = np.argsort(msp_u)                 # 낮을수록 unknown
rank_e   = np.argsort(-energy_u)             # 클수록 unknown (덜 음수)
rank_sum = np.empty_like(rank_msp)
rank_sum[rank_msp] = np.arange(len(rank_msp))
rank_sum += np.arange(len(rank_e))[np.argsort(rank_e)]  # 간단 합산

demo_unknownlike = np.argsort(rank_sum)[:N_DEMO]

# B) overconfident unknown: MSP 높은 순
demo_overconf = np.argsort(-msp_u)[:min(N_DEMO, 5)]

print("demo_unknownlike:", demo_unknownlike)
print("demo_overconf:", demo_overconf)

## Top-K 트리아지 결과를 “그림 + CSV 요약”으로 저장

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

OUT_DIR = "assets/triage"
os.makedirs(OUT_DIR, exist_ok=True)

def triage_summary(unk_i, K=5):
    labels = [ref_df.iloc[j]["label"] for j in topk_idx[unk_i][:K]]
    cnt = Counter(labels)
    top_label, top_count = cnt.most_common(1)[0]
    purity = top_count / K
    uniq = len(cnt)

    if purity >= 0.8:
        msg = f"Top-{K}: {top_label} {top_count}/{K} → '{top_label}' 계열 가능성 높음"
    elif uniq >= 4:
        msg = f"Top-{K} 라벨 다양({dict(cnt)}) → 애매 케이스(추가 확인/재측정 권장)"
    else:
        msg = f"Top-{K} 혼재({dict(cnt)}) → 후보 복수"
    return labels, purity, msg

def save_triage_case(unk_i, K=5, extra_note=""):
    unk_row = test_unknown_df.iloc[unk_i]
    labels, purity, msg = triage_summary(unk_i, K)

    plt.figure(figsize=(3*(K+1), 3))
    plt.subplot(1, K+1, 1)
    plt.imshow(unk_row["waferMap"])
    plt.title(f"UNKNOWN\n(true={unk_row['label']})")
    plt.axis("off")

    for j in range(K):
        rrow = ref_df.iloc[topk_idx[unk_i][j]]
        plt.subplot(1, K+1, j+2)
        plt.imshow(rrow["waferMap"])
        plt.title(f"{rrow['label']}\ncos={topk_sim[unk_i][j]:.3f}")
        plt.axis("off")

    title = msg + (f" | {extra_note}" if extra_note else "")
    plt.suptitle(title, y=1.05, fontsize=11)
    plt.tight_layout()

    fname = f"triage_unk{unk_i:04d}_true-{unk_row['label']}.png"
    path = os.path.join(OUT_DIR, fname)
    plt.savefig(path, dpi=200, bbox_inches="tight")
    plt.close()

    return {
        "unk_i": int(unk_i),
        "true_label": unk_row["label"],
        "topk_labels": "|".join(labels),
        "purity": float(purity),
        "note": title,
        "img_path": path
    }

def save_demo_set(indices, tag, K=5):
    rows = []
    for i in indices:
        extra = f"{tag}"
        rows.append(save_triage_case(int(i), K=K, extra_note=extra))
    return rows

# demo_unknownlike / demo_overconf가 없으면(위 셀 스킵했으면) 직접 indices를 넣어도 됨
K = 5
rows_all = []
if "demo_unknownlike" in globals():
    rows_all += save_demo_set(demo_unknownlike, "OOD-unknownlike", K=K)
if "demo_overconf" in globals():
    rows_all += save_demo_set(demo_overconf, "OOD-overconfident", K=K)

df_sum = pd.DataFrame(rows_all)

# OOD 점수도 같이 저장(있으면)
if "msp_u" in globals():
    df_sum["msp"] = df_sum["unk_i"].apply(lambda i: float(msp_u[i]))
if "energy_u" in globals():
    df_sum["energy_raw"] = df_sum["unk_i"].apply(lambda i: float(energy_u[i]))

csv_path = os.path.join(OUT_DIR, "triage_summary.csv")
df_sum.to_csv(csv_path, index=False, encoding="utf-8-sig")

print("✅ saved images to:", OUT_DIR)
print("✅ saved summary csv:", csv_path)
df_sum.head()